In [1]:
from mapepire_python import connect
import os
from textwrap import dedent
from typing import Any, Dict, Optional

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.models.ollama import Ollama
from agno.tools import tool
from dotenv import load_dotenv
from mapepire_python import connect
from pep249 import QueryParameters
from dotenv import load_dotenv


In [2]:
load_dotenv()

credentials = {
    "host": os.getenv("HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("PASSWORD"),
    "port": os.getenv("DB_PORT"),
}

ptf_currency = dedent(
    """
    With iLevel(iVersion, iRelease) AS
    (
    select OS_VERSION, OS_RELEASE from sysibmadm.env_sys_info
    )
    SELECT P.*
        FROM iLevel, systools.group_ptf_currency P
        WHERE ptf_group_release = 
            'R' CONCAT iVersion CONCAT iRelease concat '0'
        ORDER BY ptf_group_level_available -
            ptf_group_level_installed DESC
    """
)

def run_sql_statement(
    sql: str,
    parameters: Optional[QueryParameters] = None,
    creds: Dict[str, Any] = credentials,
) -> Any:
    with connect(creds) as conn:
        with conn.execute(sql, parameters=parameters) as cur:
            if cur.has_results:
                result = cur.fetchall()
                return result["data"]
            else:
                return f"No Data found for employee: {id}"

def get_ptf_currency() -> str:
    return run_sql_statement(sql=ptf_currency)

stuff = get_ptf_currency()
stuff


[{'PTF_GROUP_CURRENCY': 'UPDATE AVAILABLE',
  'PTF_GROUP_ID': 'SF99738',
  'PTF_GROUP_TITLE': 'SF99738 - 740 Group Security',
  'PTF_GROUP_LEVEL_INSTALLED': 75,
  'PTF_GROUP_LEVEL_AVAILABLE': 78,
  'LAST_UPDATED_BY_IBM': '06/10/25',
  'PTF_GROUP_RELEASE': 'R740',
  'PTF_GROUP_STATUS_ON_SYSTEM': 'INSTALLED',
  'PTF_GROUP_LAST_UPDATED_BY_IBM': '06/10/2025'},
 {'PTF_GROUP_CURRENCY': 'UPDATE AVAILABLE',
  'PTF_GROUP_ID': 'SF99739',
  'PTF_GROUP_TITLE': 'SF99739 - 740 Group Hiper',
  'PTF_GROUP_LEVEL_INSTALLED': 149,
  'PTF_GROUP_LEVEL_AVAILABLE': 152,
  'LAST_UPDATED_BY_IBM': '06/10/25',
  'PTF_GROUP_RELEASE': 'R740',
  'PTF_GROUP_STATUS_ON_SYSTEM': 'INSTALLED',
  'PTF_GROUP_LAST_UPDATED_BY_IBM': '06/10/2025'},
 {'PTF_GROUP_CURRENCY': 'UPDATE AVAILABLE',
  'PTF_GROUP_ID': 'SF99662',
  'PTF_GROUP_TITLE': 'SF99662 - 740 IBM HTTP Server for i',
  'PTF_GROUP_LEVEL_INSTALLED': 42,
  'PTF_GROUP_LEVEL_AVAILABLE': 44,
  'LAST_UPDATED_BY_IBM': '06/09/25',
  'PTF_GROUP_RELEASE': 'R740',
  'PTF_GROUP

In [3]:
services_info = dedent(
    """
    select * from qsys2.services_info 
    """
)

services = run_sql_statement(services_info)
services

[{'SERVICE_CATEGORY': 'PTF',
  'SERVICE_SCHEMA_NAME': 'QSYS2',
  'SERVICE_NAME': 'PTF_INFO',
  'SQL_OBJECT_TYPE': 'VIEW',
  'OBJECT_TYPE': '*FILE',
  'SYSTEM_OBJECT_NAME': 'PTF_INFO',
  'LATEST_DB2_GROUP_LEVEL': 26,
  'INITIAL_DB2_GROUP_LEVEL': None,
  'EARLIEST_POSSIBLE_RELEASE': 'V6R1M0',
  'EXAMPLE': "-- Description: Will an IPL impact PTF state? \r\n  SELECT PTF_IDENTIFIER, PTF_IPL_ACTION, A.* \r\n  FROM QSYS2.PTF_INFO A \r\n       WHERE PTF_IPL_ACTION <> 'NONE';"},
 {'SERVICE_CATEGORY': 'PTF',
  'SERVICE_SCHEMA_NAME': 'QSYS2',
  'SERVICE_NAME': 'GROUP_PTF_INFO',
  'SQL_OBJECT_TYPE': 'VIEW',
  'OBJECT_TYPE': '*FILE',
  'SYSTEM_OBJECT_NAME': 'GRPPTFINFO',
  'LATEST_DB2_GROUP_LEVEL': None,
  'INITIAL_DB2_GROUP_LEVEL': None,
  'EARLIEST_POSSIBLE_RELEASE': 'V6R1M0',
  'EXAMPLE': "-- Description: What is the most recently installed CUM? \r\n  SELECT MAX(PTF_GROUP_LEVEL) AS CUM_LEVEL FROM \r\n  QSYS2.GROUP_PTF_INFO P, SYSIBMADM.ENV_SYS_INFO V \r\n  WHERE PTF_GROUP_NAME = 'SF99' CONCAT V.

In [4]:
type(services[0])

dict

In [5]:
from collections import Counter
service_catagories = [item['SERVICE_CATEGORY'] for item in services]
counter = Counter(service_catagories)
dict(counter)

{'PTF': 8,
 'SECURITY': 22,
 'WORK MANAGEMENT': 35,
 'MESSAGE HANDLING': 8,
 'LIBRARIAN': 5,
 'STORAGE': 17,
 'BACKUP AND RECOVERY': 5,
 'PRODUCT': 4,
 'SPOOL': 9,
 'SYSTEM HEALTH': 4,
 'JOURNAL': 50,
 'JAVA': 3,
 'APPLICATION': 45,
 'COMMUNICATION': 21,
 'DATABASE-APPLICATION': 9,
 'DATABASE-PERFORMANCE': 10,
 'DATABASE-PLAN CACHE': 14,
 'DATABASE-UTILITY': 18,
 'MIRROR-COMMUNICATION': 8,
 'MIRROR-PRODUCT': 27,
 'MIRROR-REPLICATION': 8,
 'MIRROR-RESYNCHRONIZATION': 5,
 'MIRROR-SERVICEABILITY': 7,
 'IFS': 12,
 'MIRROR-RECLONE': 6,
 'PERFORMANCE': 1,
 'CONFIGURATION': 4,
 'MIGRATE WHILE ACTIVE': 9}

In [6]:
from collections import Counter
sql_object_types = [item['SQL_OBJECT_TYPE'] for item in services]
counter = Counter(sql_object_types)
counter

Counter({'VIEW': 129,
         'TABLE FUNCTION': 114,
         'PROCEDURE': 103,
         'SCALAR FUNCTION': 23,
         'TABLE': 4,
         'GLOBAL VARIABLE': 1})

In [7]:
from typing import Optional
from pydantic import BaseModel, Field

class ServiceInfo(BaseModel):
    SERVICE_CATEGORY: str
    SERVICE_SCHEMA_NAME: str
    SERVICE_NAME: str
    SQL_OBJECT_TYPE: str
    OBJECT_TYPE: Optional[str]
    SYSTEM_OBJECT_NAME: Optional[str]
    LATEST_DB2_GROUP_LEVEL: Optional[int] = None
    INITIAL_DB2_GROUP_LEVEL: Optional[int] = None
    EARLIEST_POSSIBLE_RELEASE: Optional[str] = None
    EXAMPLE: Optional[str] = None
    
    model_config = {
        "from_attributes": True  # For ORM compatibility
    }
    

In [8]:
service_collection = {}
for item in services:
    name = item['SERVICE_NAME']
    service_collection[name] = ServiceInfo(**item)
    
service_collection

{'PTF_INFO': ServiceInfo(SERVICE_CATEGORY='PTF', SERVICE_SCHEMA_NAME='QSYS2', SERVICE_NAME='PTF_INFO', SQL_OBJECT_TYPE='VIEW', OBJECT_TYPE='*FILE', SYSTEM_OBJECT_NAME='PTF_INFO', LATEST_DB2_GROUP_LEVEL=26, INITIAL_DB2_GROUP_LEVEL=None, EARLIEST_POSSIBLE_RELEASE='V6R1M0', EXAMPLE="-- Description: Will an IPL impact PTF state? \r\n  SELECT PTF_IDENTIFIER, PTF_IPL_ACTION, A.* \r\n  FROM QSYS2.PTF_INFO A \r\n       WHERE PTF_IPL_ACTION <> 'NONE';"),
 'GROUP_PTF_INFO': ServiceInfo(SERVICE_CATEGORY='PTF', SERVICE_SCHEMA_NAME='QSYS2', SERVICE_NAME='GROUP_PTF_INFO', SQL_OBJECT_TYPE='VIEW', OBJECT_TYPE='*FILE', SYSTEM_OBJECT_NAME='GRPPTFINFO', LATEST_DB2_GROUP_LEVEL=None, INITIAL_DB2_GROUP_LEVEL=None, EARLIEST_POSSIBLE_RELEASE='V6R1M0', EXAMPLE="-- Description: What is the most recently installed CUM? \r\n  SELECT MAX(PTF_GROUP_LEVEL) AS CUM_LEVEL FROM \r\n  QSYS2.GROUP_PTF_INFO P, SYSIBMADM.ENV_SYS_INFO V \r\n  WHERE PTF_GROUP_NAME = 'SF99' CONCAT V.OS_VERSION CONCAT V.OS_RELEASE CONCAT '0' \r

In [11]:
filtered_services = {}
for k, v in service_collection.items():
    if v.SERVICE_CATEGORY == 'SECURITY':
        print(v.model_dump())

{'SERVICE_CATEGORY': 'SECURITY', 'SERVICE_SCHEMA_NAME': 'QSYS2', 'SERVICE_NAME': 'USER_INFO', 'SQL_OBJECT_TYPE': 'VIEW', 'OBJECT_TYPE': '*FILE', 'SYSTEM_OBJECT_NAME': 'USER_INFO', 'LATEST_DB2_GROUP_LEVEL': 13, 'INITIAL_DB2_GROUP_LEVEL': None, 'EARLIEST_POSSIBLE_RELEASE': 'V6R1M0', 'EXAMPLE': "-- Description: Which users are at risk of becoming disabled due to lack of use? \r\n SELECT * FROM QSYS2.USER_INFO \r\n WHERE STATUS = '*ENABLED' AND LAST_USED_TIMESTAMP IS NOT NULL \r\n ORDER BY LAST_USED_TIMESTAMP ASC \r\n FETCH FIRST 20 ROWS ONLY;"}
{'SERVICE_CATEGORY': 'SECURITY', 'SERVICE_SCHEMA_NAME': 'QSYS2', 'SERVICE_NAME': 'FUNCTION_INFO', 'SQL_OBJECT_TYPE': 'VIEW', 'OBJECT_TYPE': '*FILE', 'SYSTEM_OBJECT_NAME': 'FCN_INFO', 'LATEST_DB2_GROUP_LEVEL': None, 'INITIAL_DB2_GROUP_LEVEL': None, 'EARLIEST_POSSIBLE_RELEASE': 'V6R1M0', 'EXAMPLE': '-- Description: Which function usage IDs exist \r\n SELECT * FROM QSYS2.FUNCTION_INFO ORDER BY FUNCTION_ID;'}
{'SERVICE_CATEGORY': 'SECURITY', 'SERVICE_S

In [16]:
import json
from typing import List

def save_services_to_json(services: List[ServiceInfo], filename: str):
    # Convert all BaseModel objects to dictionaries
    services_data = [service.model_dump() for service in services]
    for service in services:
        print(service)
    
    with open(filename, 'w') as f:
        json.dump(services_data, f, indent=2, default=str)

In [18]:
save_services_to_json(list(service_collection.values()), "service_collection.json")

SERVICE_CATEGORY='PTF' SERVICE_SCHEMA_NAME='QSYS2' SERVICE_NAME='PTF_INFO' SQL_OBJECT_TYPE='VIEW' OBJECT_TYPE='*FILE' SYSTEM_OBJECT_NAME='PTF_INFO' LATEST_DB2_GROUP_LEVEL=26 INITIAL_DB2_GROUP_LEVEL=None EARLIEST_POSSIBLE_RELEASE='V6R1M0' EXAMPLE="-- Description: Will an IPL impact PTF state? \r\n  SELECT PTF_IDENTIFIER, PTF_IPL_ACTION, A.* \r\n  FROM QSYS2.PTF_INFO A \r\n       WHERE PTF_IPL_ACTION <> 'NONE';"
SERVICE_CATEGORY='PTF' SERVICE_SCHEMA_NAME='QSYS2' SERVICE_NAME='GROUP_PTF_INFO' SQL_OBJECT_TYPE='VIEW' OBJECT_TYPE='*FILE' SYSTEM_OBJECT_NAME='GRPPTFINFO' LATEST_DB2_GROUP_LEVEL=None INITIAL_DB2_GROUP_LEVEL=None EARLIEST_POSSIBLE_RELEASE='V6R1M0' EXAMPLE="-- Description: What is the most recently installed CUM? \r\n  SELECT MAX(PTF_GROUP_LEVEL) AS CUM_LEVEL FROM \r\n  QSYS2.GROUP_PTF_INFO P, SYSIBMADM.ENV_SYS_INFO V \r\n  WHERE PTF_GROUP_NAME = 'SF99' CONCAT V.OS_VERSION CONCAT V.OS_RELEASE CONCAT '0' \r\n  AND PTF_GROUP_STATUS = 'INSTALLED';"
SERVICE_CATEGORY='PTF' SERVICE_SCHE